In [1]:
%matplotlib notebook
import pymongo
import pandas as pd
import matplotlib as plt
import json
from pandas import Series,DataFrame
pd.set_option('display.width', None)  # 设置字符显示宽度
pd.set_option('display.max_rows', None)  # 设置显示最大行
pd.set_option('display.max_columns', None)  # 设置显示最大行
#二行即可搞定画图中文乱码
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#连接数据库
client = pymongo.MongoClient('localhost',27017)
futures = client.futures2

market = futures.market
unit = futures.unit
position = futures.position

date = "20200301"
#查询会员
# BrokerID='海通期货'
BrokerID='永安期货'
#加载数据
market = DataFrame(list(market.find({'date': {'$gte': date}})))
unit = DataFrame(list(unit.find()))
position = DataFrame(list(position.find({'date': {'$gte': date}}))).dropna().drop_duplicates(['date','variety','symbol','long_party_name'], 'last')
# print(market.tail())
# #选择需要显示的字段
market = market.copy()
# 指数收盘
market['cv'] = market.apply(lambda x: x['close'] * x['open_interest'], axis=1).fillna(0)
print(market.tail())
closes = market.groupby(['date', 'variety'])[['cv', 'open_interest']].sum()
#待解决问题：根据最小价位变动，给收盘价做五舍四入处理
closes['close_index'] = closes['cv'] / closes['open_interest']
market = closes.reset_index()[['date','variety','close_index']].fillna(0)

unit=unit[[ 'variety','unit']]

# #汇总合约
data3=position[(position['long_party_name'] == BrokerID)]
data3=data3[['date','variety','long_party_name','long_openIntr']]
data3=data3.groupby(['date','variety','long_party_name'])[['long_openIntr']].sum()

data4=position[(position['short_party_name'] == BrokerID)]
data4=data4[['date','variety','short_party_name','short_openIntr']]
data4=data4.groupby(['date','variety','short_party_name'])[['short_openIntr']].sum()


# #并集
data5=pd.merge(data3,data4, on=['date','variety'],how='outer')
data5['会员简称']=data5.apply(lambda x: BrokerID,axis=1)
data5.tail(100)
#nan缺失值填充fillna()为0
data5=data5.fillna(0)
data5['净持仓']=data5.apply(lambda x: x['long_openIntr']-x['short_openIntr'],axis=1)

# df['当日涨幅']=df.apply(lambda x: x['收盘']-x['收盘'].shift(1)-1,axis=1)
#选择需要显示的字段
data5=data5[['会员简称','long_openIntr','short_openIntr','净持仓']]
data5=data5.reset_index(['variety','date'])
netpostion=data5.set_index('date')


netpostion=data5.pivot_table('净持仓',index='date',columns='variety',fill_value=0)
# print(netpostion)
#合约价值
contractValue=pd.merge(market,unit,how='left',sort=False).drop_duplicates()
contractValue['contractValue'] = contractValue.apply(lambda x: x['close_index']*x['unit'],axis=1)
contractValue=contractValue[['date','variety','contractValue']]

# #值替换replace()
# # contractValue=contractValue.replace(['TA'],'PTA')
sz=pd.merge(data5,contractValue,on=['date','variety'],how='left')
print(sz.head())
# #净持仓价值
sz['净持仓价值']=sz.apply(lambda x: int(x['净持仓']*x['contractValue']/10000 if x['contractValue']>0 else 0),axis=1)

sznet=sz[['date','variety','会员简称','净持仓价值']]
sz=sznet.sort_values(by='净持仓价值')
sz=sz.pivot_table('净持仓价值',index=['date','会员简称'],columns='variety',fill_value=0)
sz=sz.copy()
sz['化工板块']=sz[['RU','MA','V','L','PP','BU','TA']].sum(axis=1)
sz['油脂板块']=sz[['Y','P','OI']].sum(axis=1)
sz['有色板块']=sz[['CU','AL','ZN','NI','PB']].sum(axis=1)
sz['黑色板块']=sz[['RB','HC','ZC','J','JM','I']].sum(axis=1)
sz['黄金白银']=sz[['AU','AG']].sum(axis=1)
sz['工业品']=sz[['CU','AL','ZN','NI','PB','RB','FG','RU','L','V','TA','MA','PP','JM','J','ZC','I','BU','HC','SM','SF','FU']].sum(axis=1)
sz['农产品']=sz[['A','C','M','RM','Y','P','OI','CF','SR','JD','CS','AP']].sum(axis=1)
sz['商品板块']=sz.sum(axis=1)


# chemical=sz.copy()
# chemical=chemical[['RU','MA','V','L','PP','BU','TA','商品板块','工业品','化工板块']]
# chemical['max']=chemical[['RU','MA','V','L','PP','BU','TA']].idxmax(axis=1)
# chemical['min']=chemical[['RU','MA','V','L','PP','BU']].idxmin(axis=1)
# chemical['20200120':]
# chemical = chemical.reset_index()

# # futures = client.futures
# # flows = futures.chemical
# # flows.insert(json.loads(chemical.T.to_json()).values())
print("计算完毕")


                            _id  symbol      date     open     high      low  \
57023  5eb95e094222208b27184ba4  TF2009  20200511  103.345  103.400  102.765   
57024  5eb95e094222208b27184ba5  TF2012  20200511  102.770  102.770  102.190   
57025  5eb95e094222208b27184ba6  TS2006  20200511  102.430  102.520  102.265   
57026  5eb95e094222208b27184ba7  TS2009  20200511  102.015  102.135  101.860   
57027  5eb95e094222208b27184ba8  TS2012  20200511  101.835  101.925  101.680   

         close  volume  open_interest    turnover   settle  pre_settle  \
57023  102.880   15059        16911.0  1552264.42  102.950     103.395   
57024  102.295     199          590.0    20394.72  102.340     102.680   
57025  102.315    6968         9631.0  1427169.62  102.325     102.400   
57026  101.895    4308         6716.0   879062.80  101.925     102.030   
57027  101.710     270         1796.0    54974.45  101.725     101.835   

      variety           cv  
57023      TF  1739803.680  
57024      TF   

In [2]:
chemical=sz.copy()
chemical=chemical[['RU','MA','V','L','PP','BU','TA','商品板块','工业品','化工板块']]
chemical['max']=chemical[['RU','MA','V','L','PP','BU','TA']].idxmax(axis=1)
chemical['min']=chemical[['RU','MA','V','L','PP','BU']].idxmin(axis=1)
chemical['20200220':]

,variety,RU,MA,V,L,PP,BU,TA,商品板块,工业品,化工板块,max,min
date,会员简称,,,,,,,,,,,,
20200302,永安期货,-5664,-23852,147341,83288,14468,4534,-64551,2242503,386111,155564,V,MA
20200303,永安期货,-12454,-8913,156503,81057,3620,24327,-69140,2321892,403162,175000,V,RU
20200304,永安期货,-9941,-13569,156090,83222,4338,26945,-74226,3081902,548168,172859,V,MA
20200305,永安期货,-12323,-19466,158289,73947,1917,18540,-98706,3259601,558203,122198,V,MA
20200306,永安期货,-9380,-5938,163750,72220,-3226,27987,-117308,2988019,478137,128105,V,RU
20200309,永安期货,-7427,-3983,161302,57249,-168,27879,-105342,2518960,445132,129510,V,RU
20200310,永安期货,-9482,-7591,157579,60658,9469,7191,-94056,2953085,664653,123768,V,RU
20200311,永安期货,-9898,-12366,151765,49498,19975,8670,-74300,3086349,723774,133344,V,MA
20200312,永安期货,-20690,-6269,144084,44217,28614,3137,-107152,2449221,581648,85941,V,RU


In [3]:
黑色板块=sz.copy()
黑色板块=黑色板块[['RB','HC','ZC','J','JM','I','黑色板块','工业品','商品板块']] 
黑色板块['max']=黑色板块[['RB','HC','ZC','J','JM','I']].idxmax(axis=1) 
黑色板块['min']=黑色板块[['RB','HC','ZC','J','JM','I']].idxmin(axis=1)
黑色板块['20200220':]


,variety,RB,HC,ZC,J,JM,I,黑色板块,工业品,商品板块,max,min
date,会员简称,,,,,,,,,,,
20200302,永安期货,-52936,16585,-4330,-49754,29168,71121,9854,386111,2242503,I,RB
20200303,永安期货,-6479,27036,-4651,-65664,24551,68375,43168,403162,2321892,I,J
20200304,永安期货,-28264,26765,-6569,-42389,24484,211451,185478,548168,3081902,I,J
20200305,永安期货,-46098,28450,-4749,-16906,17499,243922,222118,558203,3259601,I,RB
20200306,永安期货,-37361,24827,-7000,-43902,32075,165363,134002,478137,2988019,I,J
20200309,永安期货,43766,24466,-17589,-29023,32661,83142,137423,445132,2518960,I,J
20200310,永安期货,187896,27238,-13860,-6302,23123,103050,321145,664653,2953085,RB,ZC
20200311,永安期货,277939,23943,-17018,6686,27459,120626,439635,723774,3086349,RB,ZC
20200312,永安期货,239852,44668,-27849,2722,22515,128459,410367,581648,2449221,RB,ZC


In [4]:
# import json
# from pymongo import MongoClient
# # 连接数据库
# client = MongoClient('localhost', 27017)
# db = client.futures
# flow = db.flow
# 黑色板块=sz.copy()
# 黑色板块=黑色板块[['RB','HC','ZC','J','JM','I','黑色板块','工业品','商品板块']] 
# 黑色板块['max']=黑色板块[['RB','HC','ZC','J','JM','I']].idxmax(axis=1) 
# 黑色板块['min']=黑色板块[['RB','HC','ZC','J','JM','I']].idxmin(axis=1)
# df = 黑色板块.reset_index()
# flow.insert_many(json.loads(df.T.to_json()).values())
# print("写入OK")

In [5]:
oil=sz.copy()
oil=oil[['Y','P','OI','油脂板块','农产品']]
oil['max']=oil[['Y','P','OI']].idxmax(axis=1) 
oil['min']=oil[['Y','P','OI']].idxmin(axis=1)
oil['20200220':]

,variety,Y,P,OI,油脂板块,农产品,max,min
date,会员简称,,,,,,,
20200302,永安期货,92120,12233,32881,137234,553749,Y,P
20200303,永安期货,80683,20454,27793,128930,537433,Y,P
20200304,永安期货,83854,32231,37725,153810,570731,Y,P
20200305,永安期货,108578,18485,35972,163035,608719,Y,P
20200306,永安期货,105115,-4783,35807,136139,571283,Y,P
20200309,永安期货,98543,-23439,33611,108715,597436,Y,P
20200310,永安期货,93423,-16580,28903,105746,479806,Y,P
20200311,永安期货,96851,-19569,18754,96036,444330,Y,P
20200312,永安期货,64618,-5036,29477,89059,487568,Y,P


In [6]:
有色=sz.copy()
有色=有色[['CU','AL','ZN','NI','PB','有色板块','工业品','商品板块']]
有色['max']=有色[['CU','AL','ZN','NI','PB']].idxmax(axis=1) 
有色['min']=有色[['CU','AL','ZN','NI','PB']].idxmin(axis=1)
有色['20200213':]

,variety,CU,AL,ZN,NI,PB,有色板块,工业品,商品板块,max,min
date,会员简称,,,,,,,,,,
20200302,永安期货,293437,-35545,-83502,2587,-21443,155534,386111,2242503,CU,ZN
20200303,永安期货,268912,-33230,-100258,5584,-20823,120185,403162,2321892,CU,ZN
20200304,永安期货,300631,-47737,-104068,-6032,-20746,122048,548168,3081902,CU,ZN
20200305,永安期货,302454,-44373,-122638,35046,-24006,146483,558203,3259601,CU,ZN
20200306,永安期货,297077,-36945,-110007,17982,-22639,145468,478137,2988019,CU,ZN
20200309,永安期货,209303,-36033,-109489,53050,-18229,98602,445132,2518960,CU,ZN
20200310,永安期货,230728,-35925,-104777,82796,-16563,156259,664653,2953085,CU,ZN
20200311,永安期货,205897,-32334,-98690,43197,-12078,105992,723774,3086349,CU,ZN
20200312,永安期货,158144,-28419,-95501,37665,-14912,56977,581648,2449221,CU,ZN


In [7]:
农产品=sz.copy()
农产品=农产品[['A','C','M','RM','Y','P','OI','CF','SR','JD','CS','AP','农产品']]
农产品['max']=农产品[['A','C','M','RM','Y','P','OI','CF','SR','JD','CS','AP']].idxmax(axis=1) 
农产品['min']=农产品[['A','C','M','RM','Y','P','OI','CF','SR','JD','CS','AP']].idxmin(axis=1)
农产品['20200213':]

,variety,A,C,M,RM,Y,P,OI,CF,SR,JD,CS,AP,农产品,max,min
date,会员简称,,,,,,,,,,,,,,,
20200302,永安期货,24840,148196,109198,-7693,92120,12233,32881,192062,2947,-25418,1132,-28749,553749,CF,AP
20200303,永安期货,22799,147700,134436,-14064,80683,20454,27793,176814,-1066,-31185,-544,-26387,537433,CF,JD
20200304,永安期货,22816,149765,134206,-15096,83854,32231,37725,184120,2017,-32387,-918,-27602,570731,CF,JD
20200305,永安期货,20202,149181,155769,-17735,108578,18485,35972,188796,14046,-34705,-3031,-26839,608719,CF,JD
20200306,永安期货,19734,149998,158863,-20643,105115,-4783,35807,174411,16191,-35424,-585,-27401,571283,CF,JD
20200309,永安期货,28118,158794,179674,18964,98543,-23439,33611,179826,-25507,-33593,128,-17683,597436,CF,JD
20200310,永安期货,29961,113889,162385,13586,93423,-16580,28903,131056,-28524,-31909,-1277,-15107,479806,M,JD
20200311,永安期货,26116,83605,158968,22912,96851,-19569,18754,136742,-31041,-25070,-4747,-19191,444330,M,SR
20200312,永安期货,25897,86094,164614,22990,64618,-5036,29477,184100,-37061,-21821,-7559,-18745,487568,CF,SR


In [8]:
板块=sz.copy()
板块=板块[['商品板块','工业品','化工板块','黑色板块','有色板块','农产品','油脂板块']]
板块['20200220':]

,variety,商品板块,工业品,化工板块,黑色板块,有色板块,农产品,油脂板块
date,会员简称,,,,,,,
20200302,永安期货,2242503,386111,155564,9854,155534,553749,137234
20200303,永安期货,2321892,403162,175000,43168,120185,537433,128930
20200304,永安期货,3081902,548168,172859,185478,122048,570731,153810
20200305,永安期货,3259601,558203,122198,222118,146483,608719,163035
20200306,永安期货,2988019,478137,128105,134002,145468,571283,136139
20200309,永安期货,2518960,445132,129510,137423,98602,597436,108715
20200310,永安期货,2953085,664653,123768,321145,156259,479806,105746
20200311,永安期货,3086349,723774,133344,439635,105992,444330,96036
20200312,永安期货,2449221,581648,85941,410367,56977,487568,89059
